In [ ]:
targets = {"320", "data programming", "f21", "p1", "p2", "p3", "p4", "p5", "p6", "p7", "project"}
def match(repo): 
    repo = repo.lower()
    if repo.find("cs320-lab2") == -1:
        for word in targets: 
            if repo.find(word) != -1: 
                return repo

In [ ]:
import pandas 
from github import Github

usernames = pandas.read_csv("github.csv")

g = Github() # add your OAuth token here

not_found = []
suspect = []

for username in usernames['What is your GitHub username?']: 
    try: 
        user = g.get_user(username)
        for repo in user.get_repos():
            mat = match(repo.full_name)
            if mat is not None: 
                suspect.append(mat)
    except: 
        not_found.append(username)

In [ ]:
print(not_found)

In [ ]:
for i in suspect: 
    print("https://github.com/" + i)